In [1]:
#%load_ext nb_black

In [2]:
import json
import pandas as pd
import logging
from ipywidgets import Dropdown

In [3]:
import boto3
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\pat\AppData\Local\sagemaker\sagemaker\config.yaml


In [4]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [5]:
sess = sagemaker.Session()
region = sess.boto_region_name
print(region)

us-east-1


In [6]:
# download JumpStart model_manifest file.
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

print("number of models: ", len(model_list))

number of models:  6177


In [7]:
model_df = pd.DataFrame(model_list)
model_df.sample(20)

,model_id,version,min_version,spec_key,search_keywords
3670,tensorflow-ic-bit-s-r101x1-ilsvrc2012-classifi...,2.0.0,2.80.0,community_models/tensorflow-ic-bit-s-r101x1-il...,NaN
1767,huggingface-txt2img-fictiverse-elrisitas,1.1.0,2.144.0,community_models/huggingface-txt2img-fictivers...,NaN
4892,tensorflow-ic-resnet-50-classification-1,4.0.1,2.189.0,community_models/tensorflow-ic-resnet-50-class...,NaN
3043,pytorch-eqa-bert-base-multilingual-uncased,1.2.0,2.75.0,community_models/pytorch-eqa-bert-base-multili...,NaN
3536,tensorflow-ic-bit-m-r101x1-imagenet21k-classif...,3.0.2,2.80.0,community_models/tensorflow-ic-bit-m-r101x1-im...,NaN
5778,tensorflow-spc-electra-small-1,2.0.0,2.189.0,community_models/tensorflow-spc-electra-small-...,NaN
4659,tensorflow-ic-imagenet-mobilenet-v2-075-224-cl...,3.0.0,2.80.0,community_models/tensorflow-ic-imagenet-mobile...,NaN
31,catboost-classification-model,1.2.1,2.75.0,community_models/catboost-classification-model...,NaN
2600,meta-textgeneration-llama-codellama-7b-instruct,2.3.0,2.225.0,community_models/meta-textgeneration-llama-cod...,NaN
4610,tensorflow-ic-imagenet-mobilenet-v2-050-224-cl...,4.0.2,2.189.0,community_models/tensorflow-ic-imagenet-mobile...,NaN


In [9]:
# filter-out all the Object Detection models from the manifest list.
od_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-od-" in model_id or "-od1-" in model_id) and model_id not in od_models:
        od_models.append(model_id)

print(f"Number of od models available for inference: {len(od_models)}")

Number of od models available for inference: 70


In [13]:
# display the model-ids in a dropdown to select a model for inference.
infer_model_dropdown = Dropdown(
    options=od_models,
    value="pytorch-od-nvidia-ssd",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

In [14]:
display(infer_model_dropdown)


Dropdown(description='Select a model:', index=17, layout=Layout(width='max-content'), options=('mxnet-od-faste…

In [15]:
print(infer_model_dropdown.value)

pytorch-od-nvidia-ssd


In [16]:
# filter-out all the Image Classification models from the manifest list.
ic_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-ic-" in model_id) and model_id not in ic_models:
        ic_models.append(model_id)

print(f"Number of ic models available for inference: {len(ic_models)}")

Number of ic models available for inference: 162


In [17]:
# display the model-ids in a dropdown to select a model for inference.
infer_model_dropdown = Dropdown(
    options=ic_models,
    value="pytorch-ic-alexnet",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

In [18]:
display(infer_model_dropdown)

Dropdown(description='Select a model:', layout=Layout(width='max-content'), options=('pytorch-ic-alexnet', 'py…

In [19]:
print(infer_model_dropdown.value)

pytorch-ic-alexnet
